In [ ]:
%load_ext autoreload
%autoreload 2

from coralml.visualisation.visualisation import plot_data
from coralml.constants import mapping

In [ ]:
from ds_toolbox import pretty_notebook

pretty_notebook()

In [ ]:
import cv2

In [ ]:
colour_mapping = mapping.get_colour_mapping('../data')

In [ ]:
colour_mapping

In [ ]:
name_to_colour = {'c_hard_coral_branching': (245, 185, 95),
 'c_hard_coral_submassive': (0, 0, 0),
 'c_hard_coral_boulder': (65, 50, 230),
 'c_hard_coral_encrusting': (73, 74, 74),
 'c_hard_coral_table': (78, 252, 5),
 'c_hard_coral_foliose': (186, 153, 255),
 'c_hard_coral_mushroom': (200, 103, 5),
 'c_soft_coral': (198, 5, 252),
 'c_soft_coral_gorgonian': (84, 194, 27),
 'c_sponge': (20, 145, 245),
 'c_sponge_barrel': (16, 133, 16),
 'c_fire_coral_millepora': (190, 234, 98),
 'c_algae_macro_or_leaves': (255, 233, 72)}

In [ ]:
number_to_colour = {number: name_to_colour.get(name, (0, 0,0 )) for name, number in colour_mapping.items()}

In [ ]:
colour_mapping

### Reads images and masks

In [ ]:
im = cv2.imread('../data/images_val/02_2017_0803_132449_044.jpg')
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
gt_mask = cv2.imread('../data/masks_val/02_2017_0803_132449_044_mask.png')
gt_mask = cv2.cvtColor(gt_mask, cv2.COLOR_BGR2RGB)

In [ ]:
gt_mask = cv2.cvtColor(gt_mask, cv2.COLOR_BGR2RGB)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
im2 = im

In [ ]:
non_zero = np.where(gt_mask[:, :, 0] != 0)
gt_mask[non_zero] = np.array([number_to_colour[x[0]] for x in gt_mask[non_zero]])

In [ ]:
gt_mask[non_zero]

In [ ]:
im3 = cv2.addWeighted(im2, 1, gt_mask, 0.8, 0)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(im3)
plt.grid(False)

## Evaluate

In [ ]:
from coralml.ml.evaluate import evaluate

In [ ]:
from coralml.ml.utils import load_model

In [ ]:
    model = load_model('../models/test_model/model_best.pth')

In [ ]:
model

In [ ]:
evaluate(image_file_paths=['../data/images_val/02_2017_0803_132446_043.jpg'], 
         gt_file_paths=['../data/masks_val/02_2017_0803_132446_043_mask.png'],
         model=model,
         num_classes=14,
         nn_input_size=256,
         window_sizes=[300],
         step_sizes=[300])

In [ ]:
gt_file_paths=['../data/masks_val/02_2017_0803_132446_043_mask.png']

In [ ]:
from coralml.ml import predict

In [ ]:
pred = predict.predict(image_file_paths=['../data/images_val/02_2017_0803_132446_043.jpg', '../data/images_val/02_2017_0803_132449_044.jpg'], 
                       model=model,
                       nn_input_size=256,
                       window_sizes=[300],
                       step_sizes=[300])

In [ ]:
from coralml.ml.utils import colour_mask_to_class_id_mask

In [ ]:
pred_class_id_mask = np.argmax(pred[0], axis=-1)
gt_colour_mask = cv2.imread(gt_file_paths[0])[:, :, 0]
gt_class_id_mask = colour_mask_to_class_id_mask(gt_colour_mask)

In [ ]:
classes_map = {x: y for x, y in enumerate(sorted(colour_mapping.keys()))}

In [ ]:
pred_class_id_mask

In [ ]:
def calculate_agreement(pixel_image_1, pixel_image_2, substrate_idx=None):
    """
    Given two images containing pixel annotations, calculates agreement.
    If a substrate_idx is provided, calculates IoU over that substrate,
    otherwise calculates pixel accuracy
    
    Args:
        pixel_image_1: a numpy array  
        pixel_image_2: a numpy array
        substrate_idx: a valid substrate index
    Returns:
        Agreement metric (accuracy or IoU)
    """

    intersection = ((pixel_image_1 == substrate_idx) *
                    (pixel_image_2 == substrate_idx)).sum()

    union = ((pixel_image_1 == substrate_idx) +
             (pixel_image_2 == substrate_idx)).sum()

    # If there is no pixels with that class, IoU is undefined
    return intersection, union

In [ ]:
for i in classes_map.keys():
    calculate_agreement(gt_class_id_mask, pred_class_id_mask, substrate_idx=12)

In [ ]:
classes_map

In [ ]:
from collections import defaultdict

In [ ]:
def calculate_iou_metrics_run(gt_list, pred_list, classes_map):
    """Given datasets (predicted and gt) for pixel annotations, returns IoU"""
    intersection_per_substrate = defaultdict(int)
    union_per_substrate = defaultdict(int)

    for gt, pred in zip(gt_list, pred_list):
        for substrate_idx, substrate_name in classes_map.items():
            intersection, union = \
                calculate_agreement(gt, pred, substrate_idx=substrate_idx)
            if union:
                intersection_per_substrate[substrate_name] += intersection
                union_per_substrate[substrate_name] += union

    eps = 10e-16
    # Ads epsilon to the denominator to avoid division by zero
    iou_per_substrate = {
        substrate_name: intersection_per_substrate[substrate_name] /
                        (union_per_substrate[substrate_name] + eps)
        for substrate_name in classes_map.values()
    }

    iou_average = sum(intersection_per_substrate.values()) / \
                  sum(union_per_substrate.values())

    return iou_per_substrate, iou_average

In [ ]:
calculate_iou_metrics_run([gt_class_id_mask], [pred_class_id_mask])

In [ ]:
from coralml.ml import evaluate_clef

In [ ]:
%%time

iou_per_substrate, iou_average = evaluate_clef.evaluate(
    model_path='../models/test_model/model_best.pth',
    image_file_paths=['../data/images_val/02_2017_0803_132446_043.jpg', 
                      '../data/images_val/02_2017_0803_132449_044.jpg'], 
    gt_file_paths=['../data/masks_val/02_2017_0803_132446_043_mask.png', 
                   '../data/masks_val/02_2017_0803_132449_044_mask.png'],
    nn_input_size=256, window_sizes=[500], step_sizes=[400])

In [ ]:
import os

In [ ]:
masks_dir = os.listdir('../data/masks_val')
images = [x for x in os.listdir('../data/images_val/')]
masks = [x.split('.')[0] + '_mask.png' for x in images]

In [ ]:
len(set(masks_dir).intersection(masks))

In [ ]:
# masksdir

In [ ]:
iou_average